In [65]:
import numpy as np

class LateralErrorModel:
    """
    This model describes the lateral motion of a vehicle and its response to various inputs.

    Parameters:
        self.m (float): Mass of the vehicle (kg).
        self.Vx (float): Longitudinal velocity (self.m/s).
        self.C_alpha_f (float): Front tire cornering stiffness (N/rad).
        self.C_alpha_r (float): Rear tire cornering stiffness (N/rad).
        self.l_f (float): Distance from the vehicle's center of mass to the front axle (self.m).
        self.l_r (float): Distance from the vehicle's center of mass to the rear axle (self.m).
        self.I_z (float): Moment of inertia about the vertical axis (kg self.m^2).
        self.g (float): Acceleration due to gravity (self.m/s^2).
    """
    def __init__(self, m, Vx, C_alpha_f, C_alpha_r, l_f, l_r, I_z, g):
        self.m = m
        self.Vx = Vx
        self.C_alpha_f = C_alpha_f
        self.C_alpha_r = C_alpha_r
        self.l_f = l_f
        self.l_r = l_r
        self.I_z = I_z
        self.g = g
        
    def GenerateStateSpace(self):
        A = np.array([[0, 1, 0, 0],
                           [0, -(2*self.C_alpha_f + 2*self.C_alpha_r) / (self.m * self.Vx), (2*self.C_alpha_f + 2*self.C_alpha_r) / self.m,
                            (-2*self.C_alpha_f*self.l_f + 2*self.C_alpha_r*self.l_r) / (self.m * self.Vx)],
                           [0, 0, 0, 1],
                           [0, -(2*self.l_f*self.C_alpha_f - 2*self.l_r*self.C_alpha_r) / (self.I_z * self.Vx),
                            (2*self.l_f*self.C_alpha_f - 2*self.l_r*self.C_alpha_r) / self.I_z,
                            (-2*self.l_f**2*self.C_alpha_f + 2*self.l_r**2*self.C_alpha_r) / (self.I_z * self.Vx)]])
        
        B = np.array([[0],
                           [2 * self.C_alpha_f / self.m],
                           [0],
                           [2 * self.l_f * self.C_alpha_f / self.I_z]])
        
        C = np.array([[0],
                           [(-2*self.C_alpha_f*self.l_f + 2*self.C_alpha_r*self.l_r) / (self.m * self.Vx) - self.Vx],
                           [0],
                           [(-2*self.l_f**2*self.C_alpha_f + 2*self.l_r**2*self.C_alpha_r) / (self.I_z * self.Vx)]])
        
        D = np.array([[0],
                           [self.g],
                           [0],
                           [0]])
        return A,B,C,D
    def compute_state_derivative(self, state, delta, psi_des, phi):
        """
        Computes the derivative of the state vector using the lateral error dynamics model.

        Parameters:
            state (numpy.ndarray): The current state vector [e_y, e_y_dot, e_psi, e_psi_dot].
            delta (float): The steering input.
            psi_des (float): The desired yaw rate input.
            phi (float): The roll angle input.

        Returns:
            numpy.ndarray: The derivative of the state vector [e_y_dot, e_y_ddot, e_psi_dot, e_psi_ddot].
        """
        A,B,C,D=self.GenerateStateSpace()
        state_dot = np.dot(A, state) + np.dot(B, delta) + np.dot(C, psi_des) + np.dot(D, np.sin(phi))
        return state_dot[:,0]




In [66]:
model = LateralErrorModel(m=1000.0, Vx=20.0, C_alpha_f=10000.0, C_alpha_r=15000.0, l_f=1.5, l_r=1.0, I_z=3000.0, g=9.8)

# Initial conditions and inputs
initial_state = np.array([0.0, 0.0, 0.0, 0.0])
delta_input = 0.1
psi_des_input = 0.0
phi_input = 0.1

# Compute the derivative of the state using the model instance
state_derivative = model.compute_state_derivative(initial_state, delta_input, psi_des_input, phi_input)

print("State derivative:", state_derivative)

State derivative: [0.         2.97836748 0.         1.        ]
